### BoW

テキストを数値の特徴ベクトルとして表現<br>
1,ドキュメント集合全体から単語という一意なトークンからなる語彙を作成<br>
2,各ドキュメントにおける各単語の出現回数を含んだ特徴ベクトルを作る

日本語を処理するためには、形態素解析が必要

流れ参考<br>
https://qiita.com/yasunori/items/31a23eb259482e4824e2

In [176]:
import MeCab

##形態素を品詞指定して行う
def extractKeyword(text):
    tagger = MeCab.Tagger('-Ochasen')
    tagger.parse('')
    node = tagger.parseToNode(text)
    #node = tagger.parse(text)
    keywords = []
    while node:
        if node.feature.split(",")[0] == u"名詞":
            keywords.append(node.surface)
        #elif node.feature.split(",")[0] == u"形容詞":
        #     keywords.append(node.surface)
        elif node.feature.split(",")[0] == u"動詞":
             keywords.append(node.surface)
        node = node.next
    return keywords


text = ["pythonでMeCabを使って形態素解析を行う。" , 'あなたは空腹なの？' , 'お前も空腹なのか']
vocab = []
for i in range(len(text)):
    #vocab.append(extractKeyword(text[i]))
    L = extractKeyword(text[i])
    #joined_text = ' '.join(L)
    #vocab.append(joined_text)
    vocab.append(L)

vocab


[['python', 'MeCab', '使っ', '形態素', '解析', '行う'],
 ['あなた', '空腹'],
 ['お前', '空腹', 'の']]

In [125]:
##特徴ベクトルに変換

from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
##↑引数にstop_words = ['' , '',,,,,,,]のように与える
docs = np.array(vocab)

bag = count.fit_transform(docs)

##単語ごとのid
print(count.vocabulary_)


##単語ごとのidに対する出現回数
print(bag.toarray())

{'python': 1, 'mecab': 0, '使っ': 4, '形態素': 5, '解析': 8, '行う': 7, 'あなた': 2, '空腹': 6, 'お前': 3}
[[1 1 0 0 1 1 0 1 1]
 [0 0 1 0 0 0 1 0 0]
 [0 0 0 1 0 0 1 0 0]]


In [126]:
##tfidfに変換

from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf = True , norm='l2' , smooth_idf = True)
##少数第2位まで
np.set_printoptions(precision = 2)

##tfidfの分散表現
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.41 0.41 0.   0.   0.41 0.41 0.   0.41 0.41]
 [0.   0.   0.8  0.   0.   0.   0.61 0.   0.  ]
 [0.   0.   0.   0.8  0.   0.   0.61 0.   0.  ]]


/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [127]:
tfid = tfidf.fit_transform(count.fit_transform(docs)).toarray()

/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [140]:
###最大のtfidfの値をとるインデックスを返す

m = []
for i in range(tfid.shape[0]):
    arg= np.argmax(tfid[i])
    m.append(arg)
print(m)

##mが最大tfidを取る箇所→最大単語idを取っている

[0, 2, 3]


In [ ]:
メモリ解放しておく

お試し

In [51]:
from sklearn.feature_extraction.text import CountVectorizer

def create_countvectorizer_from_texts(texts):
    count = CountVectorizer()
    docs = np.array(texts)
    bags = count.fit_transform(docs)
    print(bags.toarray())
    print(count.vocabulary_)

texts = ["He likes to play the guitar", \
"She likes to play the piano", \
"He likes to play the guitar, and she likes to play the piano"]
create_countvectorizer_from_texts(text)

[[1 0 0]
 [0 1 0]
 [0 0 1]]
{'pythonでmecabを使って形態素解析を行う': 0, 'あなたは空腹なの': 1, 'お前も空腹なのか': 2}


In [52]:
'\t'.join([str(i) for i in vocab])

"['python', 'MeCab', '使っ', '形態素', '解析', '行う']\t['あなた', '空腹']\t['お前', '空腹', 'の']"

お試し2 n-gram

https://hayataka2049.hatenablog.jp/entry/2017/02/06/215619

In [89]:
##文章をリスト化
data_lst = ["吾輩は猫である",
            "国境の長いトンネルを抜けると雪国であった",
            "恥の多い生涯を送って来ました",
            "一人の下人が、羅生門の下で雨やみを待っていた",
            "幼時から父は、私によく、金閣のことを語った"]

In [93]:
from collections import defaultdict

def ngram_split(string, n, splitter="-*-"):
    """
    string:iterableなら何でも n:n-gramのn　splitter:処理対象と混ざらなければ何でも良い
    """
    lst = []
    for i in range(len(string[:-n+1])):
        lst.append(splitter.join(string[i:i+n]))
    return lst


def itr_dict(itr):
    d = defaultdict(int)
    for x in itr:
        d[x] += 1
    return d

In [94]:
from sklearn.feature_extraction import DictVectorizer

bgram_dict_lst = [itr_dict(ngram_split(x, 3)) for x in data_lst]
dict_vectorizer = DictVectorizer()
a = dict_vectorizer.fit_transform(bgram_dict_lst).toarray()

In [95]:
a

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
        1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0.

In [96]:
len_array = np.array([[len(x)]*a.shape[1] for x in data_lst])
#↑もうちょっと綺麗な方法があるなら教えて欲しい

std_a = a/len_array

In [98]:
std_a.shape

(5, 74)

In [99]:
a.shape

(5, 74)

お試し3

In [101]:
devided_text = ['This is a pen.', 'I live in Osaka']
joined_devided_text = ' '.join(devided_text)
print(joined_devided_text)

This is a pen. I live in Osaka


word2vec

https://qiita.com/mergit/items/822dc49343c887019d44

In [ ]:
from gensim.models import word2vec

# size: 圧縮次元数
# min_count: 出現頻度の低いものをカットする
# window: 前後の単語を拾う際の窓の広さを決める
# iter: 機械学習の繰り返し回数(デフォルト:5)十分学習できていないときにこの値を調整する
# model.wv.most_similarの結果が1に近いものばかりで、model.dict['wv']のベクトル値が小さい値ばかりの 
# ときは、学習回数が少ないと考えられます。
# その場合、iterの値を大きくして、再度学習を行います。

# 事前準備したword_listを使ってWord2Vecの学習実施
model = word2vec.Word2Vec(word_list, size=100,min_count=5,window=5,iter=100)

In [162]:
waka  =pd.read_csv('wakatchi.csv',header = None)
waka = waka.rename(columns = ({0:'color'}))
waka

,color
0,red
1,Green
2,Blue


In [163]:
waka['color'][0]

'red'

In [175]:
import csv
from janome.tokenizer import Tokenizer
t = Tokenizer(wakati=True)

#with open('wakatchi.csv', encoding='utf-8') as f:
#    reader = csv.reader(f)
#    next(reader)
#    for columns in reader:
#        #for i in t.tokenize(columns[0]):
#        for i in t.tokenize(columns[0]):
#            print(i)
#        print()
        
        
for row in range(len(waka)):
    val = waka['color'][row]
    tokens = t.tokenize(val)
    for token in tokens:
        #token = token.astype()
        partOfSpeech = token.part_of_speech.split(',')#[0]
        # 今回抽出するのは名詞だけとします。（もちろん他の品詞を追加、変更、除外可能です。）
        if partOfSpeech == u'名詞':
        #    each_data.append(token.surface) # token.surfaceは表層形(語彙)です。詳しくはこちら...http://ailaby.com/janome/
        # すべての形態素を含ませたいのであればif構文を外し、each_data.append(token.surface)のみ記載してください。
            data.append(each_data)
            each_data = []
        #print(type(token))

AttributeError: 'str' object has no attribute 'part_of_speech'

In [ ]:
固有名詞
サ変接続
一般